# KGE-BERT: Inspire BERT with Knowledge Graph Embedding for Question Answering

## Set up

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets
!pip install tqdm

     |████████████████████████████████| 4.0 MB 4.1 MB/s 
     |████████████████████████████████| 6.6 MB 48.7 MB/s 
     |████████████████████████████████| 895 kB 73.9 MB/s 
     |████████████████████████████████| 596 kB 62.7 MB/s 
     |████████████████████████████████| 77 kB 8.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.2 MB/s 
     |████████████████████████████████| 136 kB 61.9 MB/s 
     |████████████████████████████████| 212 kB 84.5 MB/s 
     |████████████████████████████████| 1.1 MB 80.4 MB/s 
     |████████████████████████████████| 127 kB 90.2 MB/s 
     |████████████████████████████████| 144 kB 94.9 MB/s 
     |████████████████████████████████| 271 kB 76.5 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
import os
import torch
import sys

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'EECS487/project'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)
deepwalkPATH = os.path.join(GOOGLE_DRIVE_PATH,"/deepwalk/deepwalk")
sys.path.append(deepwalkPATH)

cuda
['squad2', '.ipynb_checkpoints', '__pycache__', 'deepwalk', 'kGraph.py', 'train_data.pickle', 'dev_data.pickle', 'dev_data_bert_large_uncased.pickle', 'train_data_bert_large_uncased.pickle', 'KGE_Dataset.py', 'model.py', 'QA_KGE_BERT.ipynb']


In [ ]:
import transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer
from transformers import BertTokenizer, BertTokenizerFast
from transformers import BertForQuestionAnswering

import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import random
import numpy as np

from transformers import AutoTokenizer

import nltk
nltk.download('punkt')


model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


# tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")
# model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")

# model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
# tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# model_list = ["roberta-base-uncased","distilroberta-base-uncased","roberta-large","bert-large-uncased","bert-base-uncased"]
# MODEL_PATH = "bert-large-uncased"
# tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

## Load SQuaD Dataset

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
raw_datasets["train"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 130319
})

## Preprocess Data

get input ids, mask ids, token_type_ids, start and end token index

In [ ]:
from KGE_Dataset import preprocess_train, preprocess_val
train_dataset = raw_datasets["train"].map(
    preprocess_train,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

val_dataset = raw_datasets["validation"].map(
    preprocess_val,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)


## Load KGE Dataset

In [ ]:
from KGE_Dataset import KGEDataset, basic_collate_fn
import pickle
# train_data = KGEDataset(train_dataset)
# dev_data = KGEDataset(val_dataset)


# Load from pickle
with open(os.path.join(GOOGLE_DRIVE_PATH,"train_data_bert_base_uncased.pickle"),"rb") as f:
    train_data = pickle.load(f)

with open(os.path.join(GOOGLE_DRIVE_PATH,"dev_data_bert_base_uncased.pickle"),"rb") as f:
    dev_data = pickle.load(f)


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import pickle
"""Save train/dev data"""
with open(os.path.join(GOOGLE_DRIVE_PATH,"train_data.pickle"),"ab") as f:
    pickle.dump(train_data,f)

with open(os.path.join(GOOGLE_DRIVE_PATH,"dev_data.pickle"),"ab") as f:
    pickle.dump(dev_data,f)


In [ ]:
train_loader = DataLoader(train_data, batch_size=10, collate_fn=basic_collate_fn, shuffle=False)
dev_loader = DataLoader(dev_data, batch_size=10, collate_fn=basic_collate_fn, shuffle=False)

In [ ]:
dev_loader = DataLoader(dev_data, batch_size=10, collate_fn=basic_collate_fn, shuffle=False)
res = next(iter(dev_loader))
for r in res:
    if type(r) == type([]):
        print(r)
    else:
        print(r.shape)

torch.Size([10, 384])
torch.Size([10, 384])
torch.Size([10, 384])
torch.Size([10, 1500])
[[49, 49, 49, 49], [37, 35, 37, 37], [73, 73, 73, 73], [81, 81, 81, 81], [164, 159, 164, 164], [0], [0], [0], [0], [205, 205, 205]]
[[49, 49, 49, 49], [40, 40, 40, 40], [77, 77, 77, 77], [82, 82, 82, 82], [165, 165, 164, 164], [0], [0], [0], [0], [209, 209, 209]]


## Grid Search

In [5]:
import itertools
from tqdm.notebook import tqdm
from model import KGEBERT, get_hyper_parameters, train_model, plot_loss
import torch.optim as optimizer
import time
import random
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_EPOCH = 11
model_type = "bert-base-uncased"

def search_param_basic():
    """Experiemnt on different hyper parameters."""
    hidden_dim, learning_rate, weight_decay = get_hyper_parameters()
    print("hidden dimension from: {}\nlearning rate from: {}\nweight_decay from: {}".format(
        hidden_dim, learning_rate, weight_decay
    ))
    best_model, best_stats = None, None
    best_metrics, best_lr, best_wd, best_hd = [0,0,0], 0, 0, 0
    for hd, lr, wd in tqdm(itertools.product(hidden_dim, learning_rate, weight_decay),
                           total=len(hidden_dim) * len(learning_rate) * len(weight_decay)):
        net = KGEBERT(model_type,hd).to(device)
        optim = optimizer.AdamW(net.parameters(),lr=lr,weight_decay=wd)
        scheduler = torch.optim.lr_scheduler.LinearLR(optim, start_factor=1.0, end_factor=0, total_iters=NUM_EPOCH)
        model, stats = train_model(net, train_loader, dev_loader, optim, scheduler,
                                   num_epoch=NUM_EPOCH, patience=6, collect_cycle=500,
                                   device=device, verbose=True)
        # print metrics
        print(f"{(hd, lr, wd)}: F1({stats['metrics'][0]}) EM({stats['metrics'][1]}) AvNA({stats['metrics'][2]})")
        # update best parameters if needed
        if stats['metrics'][0] > best_metrics[0]:
            best_metrics = stats['metrics']
            best_model, best_stats = model, stats
            best_hd, best_lr, best_wd = hd, lr, wd
    print("\n\nBest hidden dimension: {}, Best learning rate: {}, best weight_decay: {}".format(
        best_hd, best_lr, best_wd))
    print("F1: {:.4f}".format(best_stats['metrics'][0]))
    print("EM: {:.4f}".format(best_stats['metrics'][1]))
    print("AvNA: {:.4f}".format(best_stats['metrics'][2]))
    return best_model, best_stats, best_hd
best_model,best_stats,best_hd = search_param_basic()
timestr = time.strftime("%Y%m%d_%H%M%S")
print(f"Saving Model {timestr}")
torch.save(best_model, os.path.join(GOOGLE_DRIVE_PATH,f"model_saved/best_model_{best_hd}_{timestr}.pt"))
plot_loss(best_stats,timestr)

hidden dimension from: [900]
learning rate from: [2e-05]
weight_decay from: [0.02]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
------------------------ Start Training ------------------------
Epoch N